Sam Harley, Liam Doyle, Joe Demarzo

In [3]:
#https://api.arcade.academy/en/latest/tutorials/card_game/index.html This is the URL we were given after our quiz submissions. The link gave us some good inspiration into how we would define our vaeiables, as well as how we'd actually design our game.
#https://github.com/kamali1331/Python-Slot-machine/blob/main/slot%20machine.py This was a simple code that let us play within Jupyter. 
#https://github.com/Belgianwafflecorp/SlotMachine.Py This code influenced and helped us through the "rules" of our game.
#https://github.com/notaSWE/pygameslots Here we gained more knowledge on pygames. Furthermore, this code helped influence our reels.
#https://www.youtube.com/watch?v=aAJvFDSo5xQ, https://coderslegacy.com/python/python-pygame-tutorial/. These two resources really introduced us to pygames. We first watched the video, which was a coding game tutorial using pygame, and then in order to reinforce our understanding we looked at the pygame tutorial and read through it in it's entirety. These two sources were the most influential.



import pygame
import random

WIDTH, HEIGHT = 800, 500
FPS = 60

STARTING_BALANCE = 100
MIN_BET = 1

IMAGE_PATHS = {
    "cherry": "cherry.jpg",
    "lemon": "lemon.png",
    "bell": "bell.jpg",
    "star": "star.png",
    "seven": "seven.jpg"
}

SYMBOLS = list(IMAGE_PATHS.keys())

REEL_COUNT = 3
REEL_SIZE = 120
REEL_GAP = 30


def calculate_payout(reels, bet):
    if reels[0] == reels[1] == reels[2]:
        if reels[0] == "seven":
            return bet * 100
        return bet * 5
    elif len(set(reels)) == 2:
        return bet * 2
    return 0


def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Slot Machine")
    clock = pygame.time.Clock()

    images = {}
    images["cherry"] = pygame.image.load("cherry.jpeg")
    images["cherry"] = pygame.transform.scale(images["cherry"], (REEL_SIZE - 10, REEL_SIZE - 10))

    images["lemon"] = pygame.image.load("lemon.png")
    images["lemon"] = pygame.transform.scale(images["lemon"], (REEL_SIZE - 10, REEL_SIZE - 10))

    images["bell"] = pygame.image.load("bell.jpeg")
    images["bell"] = pygame.transform.scale(images["bell"], (REEL_SIZE - 10, REEL_SIZE - 10))

    images["star"] = pygame.image.load("star.png")
    images["star"] = pygame.transform.scale(images["star"], (REEL_SIZE - 10, REEL_SIZE - 10))

    images["seven"] = pygame.image.load("seven.jpeg")
    images["seven"] = pygame.transform.scale(images["seven"], (REEL_SIZE - 10, REEL_SIZE - 10))

    font_big = pygame.font.SysFont("arial", 60, bold=True)
    font_med = pygame.font.SysFont("arial", 32, bold=True)
    font_small = pygame.font.SysFont("arial", 24)

    balance = STARTING_BALANCE
    bet = 5
    reels = [random.choice(SYMBOLS) for _ in range(REEL_COUNT)]
    spinning = False
    spins_left = [0, 0, 0]
    message = ""

    total_width = REEL_COUNT * REEL_SIZE + (REEL_COUNT - 1) * REEL_GAP
    reels_x = (WIDTH - total_width) // 2
    reels_y = HEIGHT // 2 - REEL_SIZE // 2

    button_rect = pygame.Rect(WIDTH // 2 - 80, HEIGHT - 100, 160, 60)

    running = True
    while running:
        clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    bet += 1
                elif event.key == pygame.K_DOWN:
                    bet = max(MIN_BET, bet - 1)
                elif event.key == pygame.K_SPACE and not spinning:
                    if bet <= balance:
                        balance -= bet
                        spinning = True
                        spins_left = [random.randint(15, 25) + i * 5 for i in range(3)]
                        message = "Spinning..."
                    else:
                        message = "Not enough money!"

        if spinning:
            for i in range(REEL_COUNT):
                if spins_left[i] > 0:
                    reels[i] = random.choice(SYMBOLS)
                    spins_left[i] -= 1

            if all(s == 0 for s in spins_left):
                spinning = False
                win = calculate_payout(reels, bet)
                balance += win
                message = "You won $" + str(win) if win > 0 else "No win. Try again!"

        screen.fill((20, 60, 80))

        title = font_big.render("SLOT MACHINE", True, (255, 255, 255))
        screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 20))

        screen.blit(font_med.render("Balance: $" + str(balance), True, (255, 255, 255)), (20, 380))
        screen.blit(font_med.render("Bet: $" + str(bet), True, (255, 255, 255)), (20, 420))
        screen.blit(font_med.render("Up Arrow to Increase Bet", True, (255, 255, 255)), (WIDTH // 2 - title.get_width() // 2 + 20, 100))
        screen.blit(font_med.render("Down Arrow to Decrease Bet", True, (255, 255, 255)), (WIDTH // 2 - title.get_width() // 2 + 20, 140))
        screen.blit(font_med.render("Space to Spin", True, (255, 255, 255)), (525, 380))
        for i in range(REEL_COUNT):
            x = reels_x + i * (REEL_SIZE + REEL_GAP)
            rect = pygame.Rect(x, reels_y, REEL_SIZE, REEL_SIZE)
            pygame.draw.rect(screen, (5, 30, 45), rect, border_radius=10)
            pygame.draw.rect(screen, (255, 255, 255), rect, 2, border_radius=10)

            symbol = reels[i]
            img = images[symbol]
            screen.blit(img, (rect.centerx - img.get_width() // 2, rect.centery - img.get_height() // 2))

        button_color = (0, 200, 0) if not spinning else (120, 120, 120)
        pygame.draw.rect(screen, button_color, button_rect, border_radius=12)
        pygame.draw.rect(screen, (255, 255, 255), button_rect, 2, border_radius=12)
        screen.blit(font_med.render("SPIN", True, (0, 0, 0)),
                    (button_rect.centerx - 35, button_rect.centery - 17))

        msg = font_small.render(message, True, (255, 255, 0))
        screen.blit(msg, (WIDTH // 2 - msg.get_width() // 2, HEIGHT - 150))

        pygame.display.update()

    pygame.quit()


if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: No file 'cherry.jpg' found in working directory '/Users/liamdoyle/Downloads'.